In [1]:
#GymNN

import gym
import random
import numpy as np
from keras.optimizers import Adam
from statistics import mean,median
from keras.layers import Dense,Dropout
from keras.models import Sequential,load_model

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
class Config:
    lr=1e-2
    epochs=10
    steps=500
    hidden=32
    min_score=50
    gen_data=False
    game_nums=50000
    train_model=True

def init_game():
    for game in range(Config.game_nums):
        env.reset()
        for j in range(Config.steps):
            env.render(mode='human')
            act=env.action_space.sample()
            obs,r,over,info=env.step(act)
            if over:
                break

def init_players():
    data=[]
    scores=[]
    good_scores=[]
    for i in range(Config.game_nums):
        score=0
        game_mem=[]
        prev_obs=[]
        for j in range(Config.steps):
            act=random.randrange(0,2)
            obs,r,over,info=env.step(act)
            if len(prev_obs)>0:
                game_mem.append([prev_obs,act])
            prev_obs=obs
            score+=r
            if over:
                break
        if score>=Config.min_score:
            good_scores.append(score)
            for interaction in game_mem:
                data.append([interaction[0],[0,1] if interaction[1] else [0,0]])
        env.reset()
        scores.append(score)
    x=np.array(data)
    np.save('CartPoleData.npy',x)
    print('Avg Good Score: {}'.format(mean(good_scores)))

    return x

In [27]:
env=gym.make('CartPole-v0')
env.reset()
x=init_players() if Config.gen_data else np.load('CartPoleData.npy')

model=Sequential([
    Dense(input_dim=4,units=Config.hidden,activation='relu'),
    Dense(units=2,activation='softmax')
])

adam=Adam(lr=0.1)
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [28]:
if Config.train_model:
    data=np.array([i[0] for i in x[:100]])
    label=np.array([i[1] for i in x[:100]])

    model.fit(data,label,epochs=Config.epochs,batch_size=20)
    model.save('CartPole.h5')
else:
    model=load_model('CartPole.h5')

Epoch 1/10
100/100 [==============================] - 0s 3ms/step - loss: 0.3027 - acc: 0.4300
Epoch 2/10
100/100 [==============================] - 0s 78us/step - loss: 0.2984 - acc: 0.4600
Epoch 3/10
100/100 [==============================] - 0s 77us/step - loss: 0.2943 - acc: 0.4600
Epoch 4/10
100/100 [==============================] - 0s 70us/step - loss: 0.2903 - acc: 0.4700
Epoch 5/10
100/100 [==============================] - 0s 74us/step - loss: 0.2863 - acc: 0.4700
Epoch 6/10
100/100 [==============================] - 0s 86us/step - loss: 0.2824 - acc: 0.4700
Epoch 7/10
100/100 [==============================] - 0s 79us/step - loss: 0.2786 - acc: 0.4700
Epoch 8/10
100/100 [==============================] - 0s 78us/step - loss: 0.2749 - acc: 0.4700
Epoch 9/10
100/100 [==============================] - 0s 68us/step - loss: 0.2711 - acc: 0.4700
Epoch 10/10
100/100 [==============================] - 0s 79us/step - loss: 0.2674 - acc: 0.4700


In [21]:
model.get_weights()

[array([[-9.50032398e-02, -1.81157485e-01,  1.40371010e-01,
          7.58039430e-02, -1.77259237e-01,  8.50802138e-02,
         -1.67394608e-01, -1.75819527e-02, -2.85690706e-02,
          2.04051778e-01, -1.12119459e-01, -2.10362032e-01,
         -8.22711959e-02, -5.60813993e-02, -1.36371609e-02,
         -3.33097875e-02,  1.41074553e-01,  2.11212412e-01,
          1.24234103e-01,  4.00861874e-02,  4.10401635e-02,
          3.94341312e-02,  1.18441638e-02,  3.06779742e-02,
          7.64792189e-02, -2.08323896e-02,  1.47841051e-01,
         -1.49682179e-01, -1.54019013e-01,  1.26166925e-01,
         -6.57115579e-02, -1.54143706e-01, -9.83386487e-02,
         -1.53910935e-01,  2.77397875e-02,  4.77536470e-02,
          1.16018705e-01,  1.98658735e-01,  8.18798617e-02,
         -1.98296830e-01, -6.51236102e-02, -1.14903487e-01,
          1.51430503e-01, -1.18277922e-01, -6.13436848e-02,
         -2.84094345e-02, -1.94176868e-01, -8.02721009e-02,
         -3.65658253e-02,  3.61243188e-0

In [15]:
model.predict(np.array([-0.03394906, -0.14676099, -0.04547476,  0.31785848]).reshape((-1,4)))

array([[0., 1.]], dtype=float32)

In [5]:
scores=[]
choices=[]
for i in range(10):
    score=0
    game_mem=[]
    prev_obs=[]
    env.reset()
    while 1:
        env.render()
        action=random.randrange(0,2) if len(prev_obs)==0 else np.argmax(model.predict(np.array(prev_obs)))
        choices.append(action)
        obs,r,over,info=env.step(action)
        prev_obs=obs
        game_mem.append([obs,action])
        score+=r
        if over:
            break
    scores.append(score)
print('Avg Score: {}'.format(mean(scores)))

ReraisedException: Error occured while running `from pyglet.gl import *`
The original exception was:

ImportError: Library "GL" not found.

HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'. If you're running on a server, you may need a virtual frame buffer; something like this should work: 'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'